# Tensorflow MLP
Here we will attempt to design a regression model for the data. First we will get the data from the new file then select which to use. The feature vector will be {b1, a2, frac, RatioTotalArea} and the target will be {eta c}. 

There are currently 4 options:
- entire unscaled dataset: 57841
- entire scaled dataset (from unscaled): 57841
- limited dataset: 42016
- limited and scaled dataset: 42016

I will use the limited and scaled dataset to start because I can randomise easily. The data is scaled using the MaxAbsScaler to start.

### I do think that we may be loosing data on the 2nd ellipse, so another feature may be necessary. (The info for the other dimension is there, but in the 'RatioTotalArea').

## Model Thoughts
- Regression
- Output activation = Linear, Hidden Layers= ReLu because +ve inputs and outputs
- Loss: 'mean_squared_error' / 'mean_squared_logarithmic_error' = does not pinalise large values as much / 'mean_absolute_error' - more robust to outliers
- Regularizers: Penalization in cost function to prevent overfitting. There is l1 and l2 + more. 
- Optimizers:
- Dropout: Prevent overfitting. Maybe use rate of 0.2?

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import seaborn as sns
sns.set()

seed_val = 97

In [ ]:
# def split_data(dataset, seed, train_ratio=0.6, test_ratio=0.2, shuffle=True):
#     if shuffle:
#         dataset = dataset.sample(frac=1, random_state=seed).reset_index(drop=True)
#     test_train_ratio = test_ratio/(1-train_ratio)

#     train_dataset = dataset.sample(frac=train_ratio, random_state=0)
#     valid_and_test_dataset = dataset.drop(train_dataset.index)

#     test_dataset = valid_and_test_dataset.sample(frac=test_train_ratio, random_state=0)
#     validation_dataset = valid_and_test_dataset.drop(test_dataset.index)
    
#     return train_dataset, test_dataset, validation_dataset

def split_data(dataset, seed, train_ratio=0.6, shuffle=True):
    if shuffle:
        dataset = dataset.sample(frac=1, random_state=seed)#.reset_index(drop=True)

    train_dataset = dataset.sample(frac=train_ratio, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    
    return train_dataset, test_dataset

def add_bias(data):
    N1 = np.shape(data)[0]
    N2 = np.shape(data)[1]
    a = -1*np.ones((N1,N2+1))
    a[:,:-1] = data
    return a

def add_noise(dataset, target_column=4, noise_var=0.01, input_n=False, output_n=False):
    """ Called on DATAFRAME training data. """
    features = dataset.to_numpy()[:,0:target_column]
    labels = np.reshape(dataset.to_numpy()[:,target_column], (-1,1))
    
    if input_n:
        noise = np.reshape(np.random.normal(0,0.01,np.shape(features)[0]*np.shape(features)[1]),(np.shape(features)[0],np.shape(features)[1]))
        features = features + noise 

    if output_n:
        noise = np.reshape(np.random.normal(0,0.01,np.shape(labels)[0]*np.shape(labels)[1]),(np.shape(labels)[0],np.shape(labels)[1]))
        labels = labels + noise
    return features, labels

In [ ]:
name = "LIM_scaled.csv"
name = "data.csv"

dataset = pd.read_csv(name)
# data2.describe().transpose()
dataset.pop("Unnamed: 0")
dataset.describe().transpose()

Scale the input features

In [ ]:
scaled_dataset = dataset.copy()

scaled_dataset['b1'] = MaxAbsScaler().fit_transform(dataset['b1'].values.reshape(-1,1))
scaled_dataset['a2'] = MaxAbsScaler().fit_transform(dataset['a2'].values.reshape(-1,1))
scaled_dataset['RatioTotalArea'] = MaxAbsScaler().fit_transform(dataset['RatioTotalArea'].values.reshape(-1,1))
scaled_dataset['frac'] = MaxAbsScaler().fit_transform(dataset['frac'].values.reshape(-1,1))

In [ ]:
scaled_dataset.describe().transpose()

In [ ]:
train_dataset, test_dataset = split_data(scaled_dataset.copy(), seed_val, train_ratio=0.7)

sorted_train = train_dataset.sort_index()
sorted_test = test_dataset.sort_index()

Check if the training and test data represent the data.

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax[0].scatter(x=np.arange(len(sorted_train)), y=sorted_train['eta c'], marker='.', alpha=0.4)
ax[1].scatter(x=np.arange(len(sorted_test)), y=sorted_test['eta c'], marker='.', alpha=0.4)
ax[0].set_ylabel("eta c")
ax[0].set_title("Training Data")
ax[1].set_title("Testing Data")

In [ ]:
train_features = train_dataset.to_numpy()[:,0:4]
train_labels = train_dataset.to_numpy()[:,4]

# for noisey TRAINING data use this instead 
# train_features, train_labels = add_noise(train_dataset, input_n=True, output_n=True)

sorted_test = test_dataset.sort_index()
test_features = sorted_test.to_numpy()[:,0:4]
test_labels = sorted_test.to_numpy()[:,4]

## Code the Model
We have now scaled, shuffled and split the data. Have also checked that both the training and test set represent the output space. We can now move onto coding a model.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
model_number = 6

train_features = train_dataset.to_numpy()[:,0:4]
train_labels = train_dataset.to_numpy()[:,4]

# for noisey TRAINING data use this instead 
# train_features, train_labels = add_noise(train_dataset, noise_var=0.001, output_n=True)

out_nodes = 1
in_features = 4 

X = train_features.copy()
Y = train_labels.copy()

In [ ]:
# early stopping
earlystop_callback = EarlyStopping(monitor="loss", min_delta=0, patience=3, mode="min", restore_best_weights=True)

# setupt learning rate decay
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
    
# setup the optimizer
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.1, nesterov=False)

# kernel initializers for the wieghts of each layer
init_h1_kern = keras.initializers.RandomNormal(mean=0.0, stddev=0.54, seed=seed_val)
init_h2_kern = keras.initializers.RandomNormal(mean=0.0, stddev=0.54, seed=(seed_val+1))
init_out_kern = keras.initializers.RandomNormal(mean=0.0, stddev=0.54, seed=(seed_val+2))
init_bias = keras.initializers.RandomNormal(mean=0.0, stddev=0.54, seed=(seed_val+3))
# init_h1_kern = keras.initializers.Ones()
# init_h2_kern = keras.initializers.Ones()
# init_out_kern = keras.initializers.Ones()
# init_bias = keras.initializers.Zeros()


In [ ]:
# define the mlp model
model = Sequential()

# add the hidden layers and non-linear activation functions
model.add(Dense(30, input_shape=(in_features,), activation="relu", use_bias=True, kernel_initializer=init_h1_kern, bias_initializer=init_bias))
# model.add(keras.layers.Dropout(rate=0.2, seed=seed_val*10))

model.add(Dense(23, activation="relu", use_bias=True, kernel_initializer=init_h2_kern, bias_initializer=init_bias))
# keras.layers.Dropout(rate=0.2, seed=seed_val*20)

# add the output layer
# output_activation = keras.activations.relu(alpha=0.0, max_value=1.3, threshold=0)
model.add(Dense(out_nodes, activation="relu", use_bias=True, kernel_initializer=init_out_kern, bias_initializer=init_bias))


model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=["MSE","MAE", "MAPE"])

history = model.fit(X, Y, epochs=10, batch_size=500, validation_split=0.2, callbacks=[earlystop_callback])

In [ ]:
model.summary()

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
fig1 = plt.figure()
fig1, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,12))

ax[0][0].set_xlabel('Epoch')
ax[0][0].set_ylabel('MSE')
ax[0][0].plot(hist['epoch'], hist['MSE'], label='Train Error')
ax[0][0].plot(hist['epoch'], hist['val_MSE'], label='Val Error')
ax[0][0].legend()
ax[0][0].set_title("MSE Error")

ax[0][1].set_xlabel('Epoch')
ax[0][1].set_ylabel('MAE')
ax[0][1].plot(hist['epoch'], hist['MAE'], label='Train Error')
ax[0][1].plot(hist['epoch'], hist['val_MAE'], label='Val Error')
ax[0][1].legend()
ax[0][1].set_title("MAE Error")

ax[1][0].set_xlabel('Epoch')
ax[1][0].set_ylabel('MAPE')
ax[1][0].plot(hist['epoch'], hist['MAPE'], label='Mean Abs {} Error'.format("%"))
ax[1][0].legend()
ax[1][0].set_title("MAPE")

In [ ]:
test_input = test_features
test_output = np.reshape(test_labels, (-1,1))

prediction = model.predict(test_input)
accuracy = prediction-test_output

In [ ]:
fig2 = plt.figure()
plt.plot(np.arange(len(test_output)), test_output, label="TestData")
plt.plot(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4)
# plt.plot(np.arange(len(accuracy)),accuracy, label="Accuracy", alpha=0.4)
plt.legend()
plt.title("Model Prediction")

In [ ]:
fig3 = plt.figure()
plt.plot(np.arange(len(accuracy)),accuracy, label="Accuracy", alpha=0.4)
plt.legend()
plt.title("Model Accuracy")

# CHANGE MODEL NUMBER

In [ ]:
model_number = 6
model.save("NewModelTests/Regression{}".format(model_number))

## Test Model

In [ ]:
test_input = test_features
test_output = test_labels

prediction = model.predict(test_input)
accuracy = prediction-test_output


fig2 = plt.figure()
plt.plot(np.arange(len(test_output)), test_output, label="TestData")
plt.plot(np.arange(len(prediction)), prediction, label="Prediction", alpha=0.4)
# plt.plot(np.arange(len(accuracy)),accuracy, label="Accuracy", alpha=0.4)
plt.legend()
plt.title("Model Prediction")

Using a new subset of the data. 

In [ ]:
# Load the data
name = "data.csv" 
dataset = pd.read_csv(name)
dataset.pop("Unnamed: 0")

# Scale the data 
scaled_dataset = dataset.copy()
scaled_dataset['b1'] = MaxAbsScaler().fit_transform(dataset['b1'].values.reshape(-1,1))
scaled_dataset['a2'] = MaxAbsScaler().fit_transform(dataset['a2'].values.reshape(-1,1))
scaled_dataset['RatioTotalArea'] = MaxAbsScaler().fit_transform(dataset['RatioTotalArea'].values.reshape(-1,1))
scaled_dataset['frac'] = MaxAbsScaler().fit_transform(dataset['frac'].values.reshape(-1,1))

# Split the data 
# new seed value is required !=2000
train_dataset, test_dataset = split_data(scaled_dataset.copy(), 99, train_ratio=0.7)
 
# Prepare for model
# Only use 'test_features' and 'test_labels' at this stage 
train_features = train_dataset.to_numpy()[:,0:4]
train_labels = train_dataset.to_numpy()[:,4]
 
sorted_test = test_dataset.sort_index()
test_features = sorted_test.to_numpy()[:,0:4]
test_labels = sorted_test.to_numpy()[:,4]

test_input = test_features
test_output = np.reshape(test_labels, (-1,1))

In [ ]:
model_choice1 = 1
model1 = keras.models.load_model("Model/Regression{}".format(model_choice1))
model_choice2 = 2
model2 = keras.models.load_model("Model/Regression{}".format(model_choice2))

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
prediction1 = model1.predict(test_input)
accuracy1 = prediction1-test_output

prediction2 = model2.predict(test_input)
accuracy2 = prediction2-test_output

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
fig1 = plt.figure()
fig1, ax = plt.subplots(ncols=2,figsize=(18,5))

ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[0].plot(np.arange(len(prediction1)), prediction1, label="Prediction", alpha=0.4)
ax[0].set_title("Model{}".format(model_choice1))
ax[0].set_ylabel("Eta c")
ax[0].legend()

ax[1].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[1].plot(np.arange(len(prediction2)), prediction2, label="Prediction", alpha=0.4)
ax[1].set_title("Model{}".format(model_choice2))
ax[1].set_ylabel("Eta c")
ax[1].legend()
plt.close()

In [ ]:
model_choice1 = 3
model1 = keras.models.load_model("Model/Regression{}".format(model_choice1))
model_choice2 = 4
model2 = keras.models.load_model("Model/Regression{}".format(model_choice2))

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
prediction1 = model1.predict(test_input)
accuracy1 = prediction1-test_output

prediction2 = model2.predict(test_input)
accuracy2 = prediction2-test_output

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
fig2 = plt.figure()
fig2, ax = plt.subplots(ncols=2,figsize=(18,5))

ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[0].plot(np.arange(len(prediction1)), prediction1, label="Prediction", alpha=0.4)
ax[0].set_title("Model{}".format(model_choice1))
ax[0].set_ylabel("Eta c")
ax[0].legend()

ax[1].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[1].plot(np.arange(len(prediction2)), prediction2, label="Prediction", alpha=0.4)
ax[1].set_title("Model{}".format(model_choice2))
ax[1].set_ylabel("Eta c")
ax[1].legend()

plt.close()


In [ ]:
model_choice1 = 5
model1 = keras.models.load_model("Model/Regression{}".format(model_choice1))
model_choice2 = 6
model2 = keras.models.load_model("Model/Regression{}".format(model_choice2))

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
prediction1 = model1.predict(test_input)
accuracy1 = prediction1-test_output

prediction2 = model2.predict(test_input)
accuracy2 = prediction2-test_output

#_________________________________________________________________________________________________________________________________________________________________________________________________________________
fig3 = plt.figure()
fig3, ax = plt.subplots(ncols=2,figsize=(18,5))

ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[0].plot(np.arange(len(prediction1)), prediction1, label="Prediction", alpha=0.4)
ax[0].set_title("Model{}".format(model_choice1))
ax[0].set_ylabel("Eta c")
ax[0].legend()

ax[1].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[1].plot(np.arange(len(prediction2)), prediction2, label="Prediction", alpha=0.4)
ax[1].set_title("Model{}".format(model_choice2))
ax[1].set_ylabel("Eta c")
ax[1].legend()

plt.close()



In [ ]:
fig

In [ ]:
fig2

In [ ]:
fig3

## Change output layer to Relu

In [ ]:
original_model = keras.models.load_model("Model/Regression5")
original_prediction = original_model.predict(test_input)

new_model = keras.models.load_model("Model/Regression5")
new_model.layers.pop(2) 
new_model.add(Dense(out_nodes, activation="relu", use_bias=True))
new_prediction = new_model.predict(test_input)

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(ncols=2,figsize=(18,5))

ax[0].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[0].plot(np.arange(len(original_prediction)), original_prediction, label="Prediction", alpha=0.4)
ax[0].set_title("Original Model")
ax[0].set_ylabel("Eta c")
ax[0].legend()

ax[1].plot(np.arange(len(test_output)), test_output, label="TestData")
ax[1].plot(np.arange(len(new_prediction)), new_prediction, label="Prediction", alpha=0.4)
ax[1].set_title("New Model")
ax[1].set_ylabel("Eta c")
ax[1].legend()


In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(ncols=2,figsize=(18,5))

ax[0].plot(np.arange(len(accuracy1)), accuracy1, label="Prediction", alpha=0.4)
ax[0].set_title("Model1")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

ax[1].plot(np.arange(len(accuracy2)), accuracy2, label="Prediction", alpha=0.4)
ax[1].set_title("Model2")
ax[1].set_ylabel("Accuracy")
ax[1].legend()

In [ ]:
plt.plot(np.arange(len(accuracy1)), accuracy1, label="Acc1")
plt.plot(np.arange(len(accuracy2)), accuracy2, label="Acc2", alpha=0.4)
plt.title("Models 1 & 4")
plt.ylabel("Accuracy")
plt.legend()